In [49]:
from topagnps2cche1d.tools import find_extremities_binary
from scipy.signal import convolve2d
import numpy as np

In [84]:
def get_neighborhood(I, r, c):
    neighborhood = np.array(
        [
            [
                I[r - 1, c - 1] if (r - 1 >= 0) and (c - 1 >= 0) else 0,
                I[r - 1, c] if r - 1 >= 0 else 0,
                I[r - 1, c + 1] if (r - 1 >= 0) and (c + 1 <= I.shape[1]) else 0,
            ],
            [
                I[r, c - 1] if (c - 1 > 0) else 0,
                I[r, c],
                I[r, c + 1] if (c + 1 <= I.shape[1] - 1) else 0,
            ],
            [
                I[r + 1, c - 1] if (r + 1 <= I.shape[0] - 1) and (c - 1 >= 0) else 0,
                I[r + 1, c] if r + 1 <= I.shape[0] - 1 else 0,
                I[r + 1, c + 1]
                if (r + 1 <= I.shape[0] - 1) and (c + 1 <= I.shape[1])
                else 0,
            ],
        ]
    )
    return neighborhood


def find_extremities_binary(img, output_index_starts_one=False):

    neighbor_mask_d8 = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    neighbor_mask_d4 = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])

    num_neighbor_map_d8 = convolve2d(img, neighbor_mask_d8, mode="same")
    num_neighbor_map_d8 = np.where(num_neighbor_map_d8 == 0, 0, num_neighbor_map_d8)

    num_neighbor_map_d4 = convolve2d(img, neighbor_mask_d4, mode="same")
    num_neighbor_map_d4 = np.where(num_neighbor_map_d4 == 0, 0, num_neighbor_map_d4)

    count_ones_d8 = list(num_neighbor_map_d8.ravel()).count(1)
    count_ones_d4 = list(num_neighbor_map_d4.ravel()).count(1)

    if 0 < count_ones_d8 <= 2: # easily identifiable ends
        # if 1: we know one end
        # if 2: we know both ends, easiest case
        extremities = [
        (r, c) for r, c in np.transpose(np.where(num_neighbor_map_d8 == 1))]





        

    if len(extremities) == 2:
        if output_index_starts_one:
            extremities = [(r + 1, c + 1) for r, c in extremities]
        return extremities

    (rows, cols) = np.nonzero(img)

    # Initialize empty list of coordinates
    extremities = []

    # A MEDITER.................
    # For each non-zero pixel...
    for r, c in zip(rows, cols):
        neighborhood = get_neighborhood(img, r, c)
        if neighborhood == np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]]):
            extremities.append((r, c))
            break
        elif neighborhood == np.array([[1, 1, 0], [0, 1, 0], [0, 0, 0]]):
            extremities.append((r - 1, c - 1))

        elif neighborhood == np.array([[0, 1, 1], [0, 1, 0], [0, 0, 0]]):
            extremities.append((r - 1, c + 1))

        elif neighborhood == np.array([[0, 0, 0], [0, 1, 0], [0, 1, 0]]):
            extremities.append((r + 1, c))

        elif neighborhood == np.array([[0, 0, 0], [1, 1, 0], [0, 0, 0]]):
            extremities.append((r, c - 1))

        elif neighborhood == np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]]):
            extremities.append((r - 1, c - 1))

    

    return extremities

In [175]:
# I = np.array([[1, 1, 0, 0, 0],
#               [0, 1, 0, 0, 0], 
#               [0, 1, 0, 0, 0], 
#               [0, 1, 1, 0, 0],
#               [0, 0, 0, 1, 0]])

I = np.array([[1, 1, 0, 0, 0],
              [0, 1, 0, 0, 0], 
              [0, 1, 0, 0, 0], 
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 1]])

neighbor_mask_d8 = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
neighbor_mask_d4 = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])

J_d8 = convolve2d(I, neighbor_mask_d8, mode="same")
J_d8 = np.where(I == 0, 0, J_d8)
print(J_d8)

J_d4 = convolve2d(I, neighbor_mask_d4, mode="same")
J_d4 = np.where(I == 0, 0, J_d4)
print(J_d4)

# extremities = [
#     (r, c) for r, c in np.transpose(np.where((J <= min(2, J[J > 0].min())) & (I == 1)))
# ]
extremities_d8 = [
    (r, c) for r, c in np.transpose(np.where((J_d8 == 1)))
]
extremities_d4 = [
    (r, c) for r, c in np.transpose(np.where((J_d4 == 1)))
]

print(extremities_d8)
print(extremities_d4)

print(J_d8-J_d4)

# chercher les d4, tester si ils ont des voisins d8
print('It should be (0,0), and (4,3)')


[[2 2 0 0 0]
 [0 3 0 0 0]
 [0 2 0 0 0]
 [0 0 2 0 0]
 [0 0 0 2 1]]
[[1 2 0 0 0]
 [0 2 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 1]]
[(4, 4)]
[(0, 0), (2, 1), (4, 3), (4, 4)]
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 2 0 0]
 [0 0 0 1 0]]
It should be (0,0), and (4,3)


In [139]:
I = np.array([[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0]])

print(I.shape)

print(get_neighborhood(I, 3, 0))

(4, 4)
[[0 0 1]
 [0 1 1]
 [0 0 0]]


In [132]:
I[0, 1]

1

In [108]:
np.rot90(I, 4)

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0]])

In [94]:
rows, cols = np.where((J <= 2) & (I == 1))

for r, c in zip(rows, cols):
    print(r, c)

0 1
3 2


In [86]:
extremities = find_extremities_binary(I)
print(extremities)

[[0 1]]
